In [ ]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Assuming src functions are in a 'src' folder relative to the notebook ---
from src.visualization import (
    visualize_network_topology,
    visualize_grid_improvement,
    visualize_network_with_failures
)
from src.functions import (
    build_and_simplify_network,
    find_failures_with_yearly_profile,
    suggest_grid_reinforcement,
    print_analysis_results,
    update_and_save_parquet
)

print("Libraries imported successfully.")

# %%
# --- Start: Loading and Preparing the Data ---

# --- Path for original static data (network, costs) ---
original_data_path = "../blob-account-edh/challenge-data/"

# --- Path for the profiles generated by your analysis ---
results_path = "./results/data_parquet/"

print(f"\nLoading STATIC data from: {original_data_path}")
print(f"Loading GENERATED profiles from: {results_path}\n")


# Load network topology data (static input)
file_path = os.path.join(original_data_path, "250903_all_stations_anon.csv")
df_full = pd.read_csv(file_path, sep=";")
print("Successfully loaded network data.")

# --- Load all yearly profiles from your RESULTS folder ---
# Note: Using os.path.join is a more robust way to create file paths.
df_consumption = pd.read_parquet(os.path.join(results_path, "base_consumption.parquet"))
df_pv = pd.read_parquet(os.path.join(results_path, "pv_profiles.parquet"))
df_ev = pd.read_parquet(os.path.join(results_path, "ev_profiles.parquet"))
df_hp = pd.read_parquet(os.path.join(results_path, "hp_profiles.parquet"))
df_battery_in = pd.read_parquet(os.path.join(results_path, "battery_in_profiles.parquet"))
df_battery_out = pd.read_parquet(os.path.join(results_path, "battery_out_profiles.parquet"))
df_battery_soc_kwh = pd.read_parquet(os.path.join(results_path, "battery_soc_profiles.parquet"))
df_curtailed_energy = pd.read_parquet(os.path.join(results_path, "curtailed_energy_profiles.parquet"))
print("Successfully loaded all generated profiles from the results folder.")


# --- Data Preparation and Timestamp Mapping ---
# List of all dataframes that are time-series profiles
all_profiles_dfs = [
    df_consumption, df_pv, df_ev, df_hp,
    df_battery_in, df_battery_out, df_battery_soc_kwh, df_curtailed_energy
]

# Create a standard datetime index for the year 2050
num_periods = len(df_consumption.index)
datetime_index = pd.date_range(start='2050-01-01', periods=num_periods, freq='15min')

# Apply the new index to ALL time-series dataframes
for df in all_profiles_dfs:
    if df is not None:
        df.index = datetime_index

print(f"\nApplied new datetime index from {datetime_index.min()} to {datetime_index.max()} for all profiles.")


# Load reinforcement costs for comparison (static input)
df_reinforcement = pd.read_csv(os.path.join(original_data_path, "190923_Einheitskosten_Invest.csv"))
print("Successfully loaded grid reinforcement costs.")

# --- Calculate the Net Load ---
# This calculates the load at the customer's connection point, EXCLUDING battery flexibility
# Net Load = (Consumption + EV + HP) - PV Generation
# A negative net load means the customer is exporting power.
df_net_load = df_consumption.add(df_ev, fill_value=0).add(df_hp, fill_value=0).subtract(df_pv, fill_value=0)
print("\nSuccessfully calculated the combined net load profile for all customers.")

NOMINAL_VOLTAGE = 400.0

In [ ]:

# --- Calculate the Net Load ---
# This dataframe contains the final result we will plot as a line
df_net_load = (df_consumption.add(df_ev, fill_value=0)
               .add(df_hp, fill_value=0)
               .add(df_battery_in, fill_value=0) # Battery charging is a load
               .subtract(df_pv, fill_value=0)
               .subtract(df_battery_out, fill_value=0) # Battery discharging reduces load
               .add(df_curtailed_energy, fill_value=0)) # Curtailment is energy NOT sent to grid, so it increases net load from a grid perspective
               
print("\nSuccessfully calculated the combined net load profile for all customers.")

In [ ]:
# %%
# --- Start: Interactive Visualization ---

# Get list of customers from the columns of one of the dataframes
customer_list = df_consumption.columns.tolist()

# Define widgets
customer_dropdown = widgets.Dropdown(
    options=customer_list,
    value=customer_list[0],
    description='Customer:',
    style={'description_width': 'initial'}
)

week_selector = widgets.Dropdown(
    options=list(range(1, 53)), # Year has 52 weeks
    value=28, # A summer week to show PV generation
    description='Week of Year:',
    style={'description_width': 'initial'}
)

# Create an output widget to hold the plot
plot_output = widgets.Output()

# Main function to update and display the plot
def update_plot(change):
    # Get selected values   
    customer = customer_dropdown.value
    week = week_selector.value

    # Filter data for the selected week
    # isocalendar() returns a df with year, week, and day. We select by week.
    weekly_mask = (df_consumption.index.isocalendar().week == week)

    # Prepare data slices for the selected customer and week
    time_axis = df_consumption[weekly_mask].index
    
    # Load components (positive values)
    cons = df_consumption.loc[weekly_mask, customer]
    ev = df_ev.loc[weekly_mask, customer]
    hp = df_hp.loc[weekly_mask, customer]
    batt_in = df_battery_in.loc[weekly_mask, customer]
    
    # Generation components (we will plot their negative values)
    pv = df_pv.loc[weekly_mask, customer]
    batt_out = df_battery_out.loc[weekly_mask, customer]
    curtail = df_curtailed_energy.loc[weekly_mask, customer]

    # Final Net Load
    net_load = df_net_load.loc[weekly_mask, customer]

    # Create the figure
    fig = go.Figure()

    # --- Add traces for LOAD components (plotted as positive stacked areas) ---
    fig.add_trace(go.Scatter(
        x=time_axis, y=cons, name='Consumption',
        mode='lines', stackgroup='positive', line=dict(width=0.5, color='royalblue'),
        hovertemplate='Time: %{x}<br>Consumption: %{y:.2f} kW'
    ))
    fig.add_trace(go.Scatter(
        x=time_axis, y=ev, name='EV Charging',
        mode='lines', stackgroup='positive', line=dict(width=0.5, color='darkorange'),
        hovertemplate='Time: %{x}<br>EV Charging: %{y:.2f} kW'
    ))
    fig.add_trace(go.Scatter(
        x=time_axis, y=hp, name='Heat Pump',
        mode='lines', stackgroup='positive', line=dict(width=0.5, color='firebrick'),
        hovertemplate='Time: %{x}<br>Heat Pump: %{y:.2f} kW'
    ))
    fig.add_trace(go.Scatter(
        x=time_axis, y=batt_in, name='Battery Charging',
        mode='lines', stackgroup='positive', line=dict(width=0.5, color='mediumpurple'),
        hovertemplate='Time: %{x}<br>Battery Charging: %{y:.2f} kW'
    ))

    # --- Add traces for GENERATION/SUPPLY components (plotted as negative stacked areas) ---
    fig.add_trace(go.Scatter(
        x=time_axis, y=-pv, name='PV Generation', # Negative value
        mode='lines', stackgroup='negative', line=dict(width=0.5, color='gold'),
        hovertemplate='Time: %{x}<br>PV Generation: %{y:.2f} kW'
    ))
    fig.add_trace(go.Scatter(
        x=time_axis, y=-batt_out, name='Battery Discharging', # Negative value
        mode='lines', stackgroup='negative', line=dict(width=0.5, color='limegreen'),
        hovertemplate='Time: %{x}<br>Battery Discharging: %{y:.2f} kW'
    ))
    fig.add_trace(go.Scatter(
        x=time_axis, y=-curtail, name='PV Curtailment', # Negative value
        mode='lines', stackgroup='negative', line=dict(width=0.5, color='lightgrey'),
        hovertemplate='Time: %{x}<br>Curtailment: %{y:.2f} kW'
    ))

    # --- Add the FINAL NET LOAD as a prominent line on top ---
    fig.add_trace(go.Scatter(
        x=time_axis, y=net_load, name='Total Net Final Load',
        mode='lines', line=dict(color='black', width=3),
        hovertemplate='Time: %{x}<br><b>Net Load</b>: %{y:.2f} kW'
    ))

    # Update layout
    fig.update_layout(
        title=f"<b>Load Profile Breakdown for {customer} - Week {week}</b>",
        xaxis_title="Date and Time",
        yaxis_title="Power (kW)",
        legend_title="Load Components",
        hovermode="x unified",
        template="plotly_white"
    )
    # Add a zero line for reference
    fig.add_hline(y=0, line_width=1, line_dash="dash", line_color="grey")


    # Clear previous output and display the new plot
    with plot_output:
        clear_output(wait=True)
        display(fig);

# Link the widgets to the update function
customer_dropdown.observe(update_plot, names='value')
week_selector.observe(update_plot, names='value')

# Display the user interface
# Arrange controls horizontally
controls = widgets.HBox([customer_dropdown, week_selector])
# Display controls and the plot output area vertically
display(widgets.VBox([controls, plot_output]))

# Trigger the first plot draw manually
update_plot(None)

In [ ]:
def evaluate_trilemma(
    station_name: str,
    leg_customer_ids: list,
    df_full_topology: pd.DataFrame,
    all_profiles: dict,
    df_net_load_full_station: pd.DataFrame,
    df_reinforcement_costs: pd.DataFrame,
):
    """
    Evaluates the energy trilemma for a given Local Energy Community (LEG).

    The trilemma is assessed based on three key metrics:
    1. Grid Reinforcement Cost: The cost to upgrade the shared grid infrastructure
       of the station to handle the total load, including the LEG's contribution.
    2. Installed PV Capacity: The total peak power (kWp) of all PV systems
       within the specified LEG.
    3. Autarchy: The self-sufficiency rate of the LEG, calculated as the ratio of
       self-consumed energy to the total energy consumed by the LEG.

    Args:
        station_name (str): The name of the station the LEG belongs to.
        leg_customer_ids (list): A list of customer IDs (e.g., 'HAS-xxxx') forming the LEG.
        df_full_topology (pd.DataFrame): The complete network topology data (`df_full`).
        all_profiles (dict): A dictionary containing all profile DataFrames
                             (e.g., {"Consumption": df_consumption, "PV": df_pv, ...}).
        df_net_load_full_station (pd.DataFrame): The pre-calculated net load for the entire station.
        df_reinforcement_costs (pd.DataFrame): DataFrame with reinforcement costs.

    Returns:
        dict: A dictionary containing the trilemma results:
              {'grid_reinforcement_cost_chf': float,
               'installed_pv_kwp': float,
               'autarchy_percentage': float}
    """
    print("\n" + "="*80)
    print(f"--- Starting Trilemma Evaluation for LEG in Station: '{station_name}' ---")
    print(f"LEG consists of {len(leg_customer_ids)} members.")
    print("="*80 + "\n")
    
    # ==========================================================================
    # --- 1. Calculate Grid Reinforcement Cost for the Entire Station ---
    # The cost depends on the total load of the station, as infrastructure is shared.
    # ==========================================================================
    print(f"--- [1/3] Evaluating Grid Reinforcement Cost for Station: '{station_name}' ---")
    df_one_station = df_full_topology[df_full_topology['station'] == station_name].copy()
    
    # Build the network graph for the station
    G, consumer_props, roots = build_and_simplify_network(df_one_station)
    
    print("\nRunning dynamic profile analysis to find failures...")
    dynamic_results = find_failures_with_yearly_profile(
        graph=G,
        net_profile_df=df_net_load_full_station,
        consumer_props=consumer_props,
        root_node_ids=roots,
        nominal_voltage=NOMINAL_VOLTAGE
    )
    
    print("\nCalculating suggested grid reinforcements...")
    reinforcement_results = suggest_grid_reinforcement(
        initial_graph=G,
        initial_results=dynamic_results,
        reinforcement_costs_df=df_reinforcement_costs,
        net_profile_df=df_net_load_full_station,
        consumer_props=consumer_props,
        root_node_ids=roots,
        nominal_voltage=NOMINAL_VOLTAGE
    )
    
    grid_cost = reinforcement_results.get('total_cost_CHF', 0.0)
    print(f"✅ Finished Grid Cost Calculation.")
    print(f"--> Estimated Grid Reinforcement Cost: {grid_cost:.2f} CHF\n")

    # ==========================================================================
    # --- 2. Calculate Total Installed PV Capacity for the LEG ---
    # ==========================================================================
    print(f"--- [2/3] Calculating Installed PV Capacity for the LEG ---")
    df_pv_profiles = all_profiles.get("PV")
    
    # Filter for LEG customers that actually have a PV profile
    leg_pv_customers = [cid for cid in leg_customer_ids if cid in df_pv_profiles.columns]
    
    if not leg_pv_customers:
        installed_pv = 0.0
    else:
        # Get the max value (peak power in kW) for each customer and sum them up
        installed_pv = df_pv_profiles[leg_pv_customers].max().sum()
        
    print(f"✅ Finished PV Capacity Calculation.")
    print(f"--> Total Installed PV in LEG: {installed_pv:.2f} kWp\n")

    # ==========================================================================
    # --- 3. Calculate Autarchy (Self-Sufficiency) for the LEG ---
    # ==========================================================================
    print(f"--- [3/3] Calculating Autarchy for the LEG ---")

    def get_leg_profile(profile_df, default_val=0):
        """Helper to safely get and sum profiles for LEG members."""
        if profile_df is None:
            # This should not happen if all_profiles is complete, but it's a safe fallback.
            return pd.Series(default_val, index=all_profiles["Consumption"].index)
        
        # Find which of the LEG customers are in this dataframe's columns
        leg_cols = [cid for cid in leg_customer_ids if cid in profile_df.columns]
        
        if not leg_cols:
            return pd.Series(default_val, index=profile_df.index)
            
        return profile_df[leg_cols].sum(axis=1)

    # Calculate total load and generation profiles specifically for the LEG
    total_load_profile_kw = (get_leg_profile(all_profiles.get("Consumption")) +
                             get_leg_profile(all_profiles.get("EV")) +
                             get_leg_profile(all_profiles.get("HP")) +
                             get_leg_profile(all_profiles.get("battery_in")))

    total_generation_profile_kw = (get_leg_profile(all_profiles.get("PV")) +
                                   get_leg_profile(all_profiles.get("battery_out")))

    # Convert power (kW) to energy (kWh) by multiplying by the time interval in hours (15 min = 0.25 h)
    time_interval_h = 0.25
    total_energy_consumed_kwh = total_load_profile_kw.sum() * time_interval_h
    
    if total_energy_consumed_kwh == 0:
        autarchy_percentage = 0.0
        print("-> Total energy consumption for the LEG is zero. Autarchy is 0%.")
    else:
        # Self-consumed power at each timestep is the minimum of what was generated and what was loaded
        self_consumption_profile_kw = np.minimum(total_load_profile_kw, total_generation_profile_kw)
        
        # Calculate total self-consumed energy
        total_self_consumed_energy_kwh = self_consumption_profile_kw.sum() * time_interval_h
        
        # Autarchy = (Energy self-consumed / Total energy consumed)
        autarchy = total_self_consumed_energy_kwh / total_energy_consumed_kwh
        autarchy_percentage = autarchy * 100
        print(f"-> LEG Total Yearly Consumption: {total_energy_consumed_kwh:,.2f} kWh")
        print(f"-> LEG Self-Consumed Energy:   {total_self_consumed_energy_kwh:,.2f} kWh")
        
    print(f"✅ Finished Autarchy Calculation.")
    print(f"--> Autarchy of the LEG: {autarchy_percentage:.2f} %\n")
    
    # ==========================================================================
    # --- 4. Calculate Total Emissions for the LEG ---
    # ==========================================================================
    # Assumptions:
    # - PV/battery-out energy: 10 kg CO2/kWh
    # - Grid electricity: 37 kg CO2/kWh (Swiss mix: https://ourworldindata.org/grapher/carbon-intensity-electricity)
    grid_emission_factor = 37  # gCO2 per kWh
    solar_emission_factor = 10  # gCO2 per kWh

    # Energy bought from grid = total consumption - self-consumed (PV+battery)
    energy_from_grid_kwh = total_energy_consumed_kwh - total_self_consumed_energy_kwh if total_energy_consumed_kwh > 0 else 0.0
    leg_emiss_factor_CO2KWh = (energy_from_grid_kwh * grid_emission_factor + total_self_consumed_energy_kwh * solar_emission_factor) / total_energy_consumed_kwh
    print(f"✅ Finished Emissions Calculation.")
    print(f"--> Ave. LEG Emission Factor for Electricity: {leg_emiss_factor_CO2KWh:,.2f} gCO2 / kWh\n")


    # --- Return the final results ---
    final_results = {
        'grid_reinforcement_cost_chf': grid_cost,
        'installed_pv_kwp': installed_pv,
        'autarchy_percentage': autarchy_percentage,
        'leg_emiss_factor_CO2KWh': leg_emiss_factor_CO2KWh
    }
    
    # ==========================================================================
    # --- 5. Calculate Revenue for the whole LEG ---
    # ==========================================================================
    # Assumptions:
    grid_cost_per_kwh = 0.33  # CHF/kWh  (what user pays for grid import) ## https://www.prix-electricite.elcom.admin.ch/map?period=2024&category=H2
    feed_in_tariff_per_kwh = 0.06  # CHF/kWh (what user earns for export)


    # Energy exported to grid (PV+battery generation not self-consumed)
    energy_exported_profile_kw = total_generation_profile_kw - self_consumption_profile_kw
    energy_exported_profile_kw[energy_exported_profile_kw < 0] = 0  # Only positive exports
    total_energy_exported_kwh = energy_exported_profile_kw.sum() * time_interval_h

    # Revenue/cost calculation
    total_grid_energy_cost_chf = energy_from_grid_kwh * grid_cost_per_kwh
    total_feed_in_revenue_chf = total_energy_exported_kwh * feed_in_tariff_per_kwh
    # net_revenue_chf_year = total_feed_in_revenue_chf - total_grid_energy_cost_chf

    # load battery specs from file
    battery_specs_path = f"./results/battery_specs_station_{station_name}.csv"
    battery_specs_df = pd.read_csv(battery_specs_path, index_col='customer_id')
    total_battery_capacity_kwh = battery_specs_df['capacity_kwh'].sum()

    # installation cost of PV
    installation_cost_per_kw_PV = 1500  # CHF per kWp installed ## https://en.houzy.ch/solar-calculator
    installation_cost_per_kwh_battery = 400  # CHF per kWh of battery capacity ## rough estimate
    pv_cost = installed_pv * installation_cost_per_kw_PV
    battery_cost = total_battery_capacity_kwh * installation_cost_per_kwh_battery  
    installation_cost = pv_cost + battery_cost


    print(f"✅ Finished Revenue Calculation.")

    # Time to ROI calculation -- defined comparing LEG with baseline, that assumes no PV installed
    time_to_roi_years = installation_cost / total_feed_in_revenue_chf 
    print(f"-> Estimated Time to ROI: {time_to_roi_years:.2f} years")
    
    print(f"--> Grid Cost: {total_grid_energy_cost_chf:,.2f} CHF")
    print(f"--> Feed-in Revenue: {total_feed_in_revenue_chf:,.2f} CHF")
    print(f"--> Installation Cost of PV: {pv_cost:,.2f} CHF")
    print(f"--> Installation Cost of Battery: {battery_cost:,.2f} CHF")
    
    # --- Return the final results ---
    final_results = {
        'grid_reinforcement_cost_chf': grid_cost,
        'installed_pv_kwp': installed_pv,
        'autarchy_percentage': autarchy_percentage,
        'leg_emiss_factor_CO2KWh': leg_emiss_factor_CO2KWh,
        'total_feed_in_revenue_chf': total_feed_in_revenue_chf,
        'time_to_roi_years': time_to_roi_years 
    }

    
    print("="*80)
    print("--- Trilemma Evaluation Complete ---")
    print(f"Grid Reinforcement Cost: {final_results['grid_reinforcement_cost_chf']:,.2f} CHF")
    print(f"LEG Installed PV:        {final_results['installed_pv_kwp']:.2f} kWp")
    print(f"LEG Autarchy:            {final_results['autarchy_percentage']:.2f} %")
    # print(f"LEG average emission factor:            {final_results['leg_emiss_factor_CO2KWh']:.2f} gCO2/kWh")
    # print(f"LEG net revenue CHF:            {final_results['net_revenue_chf_year']:.2f} CHF")
    print("="*80)

    return final_results

In [ ]:
# --- Example: Evaluating the Trilemma for a Sample LEG ---

# 1. First, create a more comprehensive dictionary of all profiles for easy access
all_profiles_data = {
    "Consumption": df_consumption, 
    "PV": df_pv, 
    "EV": df_ev, 
    "HP": df_hp,
    "battery_in": df_battery_in,
    "battery_out": df_battery_out
}

# 2. Define our sample LEG
# Let's pick a station and create a LEG from a subset of its customers.

# --- FIX APPLIED HERE ---
# Instead of relying on the interactive widget, we will hardcode a station name.
# This makes the example runnable on its own.
# You can change "ST-25662" to any other valid station name from your data.
if not df_full.empty:
    selected_station_name = df_full['station'].unique()[0] # Programmatically select the first available station
    print(f"Creating a sample LEG for the station: '{selected_station_name}'")
else:
    print("Station data is empty. Cannot run the example.")
    selected_station_name = None

# Proceed only if we have a valid station name
if selected_station_name:
    # Find all unique customer connection points ('HAS-xxxx') for this station
    station_data = df_full[df_full['station'] == selected_station_name]
    all_customers_in_station = sorted([
        node for node in station_data['from'].unique() if str(node).startswith('HAS')
    ])

    # Let's create a LEG with the first 10 customers in this station who have PV installed.
    pv_customers_in_station = [
        c for c in all_customers_in_station if c in df_pv.columns
    ]
    if len(pv_customers_in_station) > 10:
        sample_leg_customers = pv_customers_in_station[:10]
    else:
        # If there are fewer than 10 PV customers, take all of them
        sample_leg_customers = pv_customers_in_station

    if not sample_leg_customers:
        print(f"\nCould not create a sample LEG for station '{selected_station_name}' as no customers with PV profiles were found.")
    else:
        # 3. Run the trilemma evaluation function
        # Note: We pass `df_net_load` which is the net load for the *entire dataset*,
        # The function will internally handle filtering for the specific station's needs.
        trilemma_results = evaluate_trilemma(
            station_name=selected_station_name,
            leg_customer_ids=sample_leg_customers,
            df_full_topology=df_full,
            all_profiles=all_profiles_data,
            df_net_load_full_station=df_net_load,
            df_reinforcement_costs=df_reinforcement
        )

In [ ]:
import pandas as pd
import warnings

# Suppress potential warnings from the analysis functions for a cleaner output during the loop
warnings.filterwarnings('ignore', category=FutureWarning)

# --- 1. Prepare for Batch Analysis ---

# Create the dictionary of all profiles needed for the function
all_profiles_data = {
    "Consumption": df_consumption, 
    "PV": df_pv, 
    "EV": df_ev, 
    "HP": df_hp,
    "battery_in": df_battery_in,
    "battery_out": df_battery_out
}

# Get a list of all unique station names to iterate over
all_station_names = sorted(df_full['station'].unique())
total_stations = len(all_station_names)
print(f"Found {total_stations} unique stations to analyze.")

all_results = []

# --- 2. Loop Through Each Station and Evaluate ---

for i, station_name in enumerate(all_station_names):
    print(f"\n--- [{i+1}/{total_stations}] Processing Station: {station_name} ---")
    
    try:
        # For this analysis, the LEG is defined as ALL customers in the station
        station_data = df_full[df_full['station'] == station_name]
        leg_customer_ids = sorted([
            node for node in station_data['from'].unique() if str(node).startswith('HAS')
        ])

        if not leg_customer_ids:
            print(f"-> Skipping station {station_name}: No customers ('HAS-xxxx') found.")
            continue

        # Run the trilemma evaluation function
        trilemma_results = evaluate_trilemma(
            station_name=station_name,
            leg_customer_ids=leg_customer_ids,
            df_full_topology=df_full,
            all_profiles=all_profiles_data,
            df_net_load_full_station=df_net_load,
            df_reinforcement_costs=df_reinforcement
        )
        
        # Add the station name to the results and store it
        trilemma_results['station_name'] = station_name
        all_results.append(trilemma_results)

    except Exception as e:
        print(f"-> 🚨 An error occurred while processing station {station_name}: {e}")
        print("-> Skipping this station and continuing.")

    print('\n 🚨 debug: we are stopping at station 1')
    break  # Remove or comment out this line to process all stations

# --- 3. Consolidate Results into a DataFrame ---

if not all_results:
    print("\nNo stations were successfully analyzed. Cannot create plot.")
else:
    results_df = pd.DataFrame(all_results)
    # Reorder columns for better readability
    results_df = results_df[['station_name', 'grid_reinforcement_cost_chf', 'installed_pv_kwp', 'autarchy_percentage']]

    print("\n\n" + "="*80)
    print("--- Trilemma Evaluation for All Stations: Complete ---")
    print("="*80)
    display(results_df)

In [ ]:
import plotly.graph_objects as go
import numpy as np

if 'results_df' in locals() and not results_df.empty:
    
    # Create the hover text to show all relevant information
    hover_text = [
        f"<b>Station:</b> {row.station_name}<br>"
        f"<b>Grid Cost:</b> {row.grid_reinforcement_cost_chf:,.0f} CHF<br>"
        f"<b>Autarchy:</b> {row.autarchy_percentage:.1f}%<br>"
        f"<b>Installed PV:</b> {row.installed_pv_kwp:.1f} kWp"
        for row in results_df.itertuples()
    ]

    fig = go.Figure(data=[go.Scatter(
        x=results_df['grid_reinforcement_cost_chf'],
        y=results_df['autarchy_percentage'],
        mode='markers',
        text=results_df['station_name'], # Text for hover
        hoverinfo='text',
        hovertext=hover_text,
        marker=dict(
            # Size of the bubble is proportional to Installed PV
            size=results_df['installed_pv_kwp'],
            # We add a minimum size to see stations with 0 PV
            sizemin=4,
            # This scales the bubble sizes to be reasonable on the plot
            sizeref=np.max(results_df['installed_pv_kwp']) / 50**2,
            sizemode='area',
            # Color of the bubble is also based on Installed PV for emphasis
            color=results_df['installed_pv_kwp'],
            colorscale='Viridis', # A nice color scale
            showscale=True,
            colorbar=dict(
                title="Installed PV (kWp)"
            )
        )
    )])

    fig.update_layout(
        title='<b>Energy Trilemma Analysis Across All Stations</b><br><sup>Lower Cost, Higher Autarchy, and Larger Bubbles are Better</sup>',
        xaxis=dict(
            title='Grid Reinforcement Cost (CHF)',
            gridcolor='lightgrey',
            type='log' # Use a log scale if costs vary wildly, otherwise 'linear'
        ),
        yaxis=dict(
            title='Autarchy (Self-Sufficiency %)',
            gridcolor='lightgrey'
        ),
        plot_bgcolor='white',
        height=700,
        legend_title_text='Metrics'
    )

    fig.show()
else:
    print("Cannot create plot because the results DataFrame is empty or does not exist.")